In [4]:
import os

In [5]:
%pwd

'/Users/amoghagadde/Desktop/Amogha/Projects/Data_Science/Text-Summarizer/research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'/Users/amoghagadde/Desktop/Amogha/Projects/Data_Science/Text-Summarizer'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [9]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [11]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

[2024-08-04 01:00:55,153: INFO: config: PyTorch version 2.3.1 available.]


In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [15]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
    
except Exception as e:
    raise e

[2024-08-04 01:07:09,435: INFO: common: yaml file: /Users/amoghagadde/Desktop/Amogha/Projects/Data_Science/Text-Summarizer/config/config.yaml loaded successfully]
[2024-08-04 01:07:09,443: INFO: common: yaml file: /Users/amoghagadde/Desktop/Amogha/Projects/Data_Science/Text-Summarizer/params.yaml loaded successfully]
[2024-08-04 01:07:09,444: INFO: common: created directory at: artifacts]
[2024-08-04 01:07:09,445: INFO: common: created directory at: /Users/amoghagadde/Desktop/Amogha/Projects/Data_Science/Text-Summarizer/artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/Users/amoghagadde/Desktop/Amogha/Jupyter/anaconda3/envs/textS/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]